In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

plt.style.use('fivethirtyeight')
sns.set_context("notebook")

Now let's play around a bit with the large baby names dataset we saw in lecture 1. We'll start by loading that dataset from the social security administration's website.

To keep the data small enough to avoid crashing datahub, we're going to look at only California rather than looking at the national dataset.

In [3]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "babynamesbystate.zip"
if not os.path.exists(local_filename): # if the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'STATE.CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.sample(5)

,State,Sex,Year,Name,Count
390650,CA,M,2017,Issac,84
370074,CA,M,2010,Dante,169
382103,CA,M,2014,Carlo,43
386316,CA,M,2015,Hardy,8
399626,CA,M,2020,Cassian,32


## Goal 1: Find the most popular baby name in California in 2018

In [5]:
babynames[babynames["Year"] == 2018].sort_values("Count",ascending=False).head(5)

,State,Sex,Year,Name,Count
221194,CA,F,2018,Emma,2751
393150,CA,M,2018,Noah,2572
221195,CA,F,2018,Mia,2504
221196,CA,F,2018,Olivia,2472
393151,CA,M,2018,Liam,2415


## Goal 2: Baby names that start with j. 

In [7]:
babynames[babynames["Name"].str.startswith("J")]

,State,Sex,Year,Name,Count
16,CA,F,1910,Josephine,66
44,CA,F,1910,Jean,35
46,CA,F,1910,Jessie,32
59,CA,F,1910,Julia,28
66,CA,F,1910,Juanita,25
...,...,...,...,...,...
407245,CA,M,2022,Jibreel,5
407246,CA,M,2022,Joseangel,5
407247,CA,M,2022,Josejulian,5
407248,CA,M,2022,Juelz,5


## Goal 3: Sort names by their length.

Approach 1a: Create a new series of only the lengths. Then add that series to the dataframe as a column. Then sort by that column. Then drop that column.

In [10]:
babyname_lengths = babynames["Name"].map(len)
# frame = length.to_frame()
# frame.sort_values("Name")

#add that series to the dataframe as a column
babynames["name_length"] = babyname_lengths

#sort by that column
babynames_by_length = babynames.sort_values(by = "name_length")

#drop that column
babynames_by_length = babynames_by_length.drop("name_length", axis=1)
babynames_by_length.head(5)

,State,Sex,Year,Name,Count
326570,CA,M,1993,An,8
292150,CA,M,1976,Al,13
252556,CA,M,1937,Al,21
401470,CA,M,2020,Jr,5
260022,CA,M,1948,Ed,43


Approach 1b: Same as 1a, but use str.len() to generate the lengths of the strings.

In [ ]:
babyname_lengths = babynames["Name"].str.len()

Approach 2: Generate an index that is in the order we want. Pass that index to loc.

In [11]:
babynames.loc[babynames["Name"].str.len().sort_values().index].head(5)

,State,Sex,Year,Name,Count,name_length
326570,CA,M,1993,An,8,2
292150,CA,M,1976,Al,13,2
252556,CA,M,1937,Al,21,2
401470,CA,M,2020,Jr,5,2
260022,CA,M,1948,Ed,43,2


How does this work exactly? Let's break it into pieces.

In [12]:
lengths_sorted_by_length = babynames["Name"].str.len().sort_values()
lengths_sorted_by_length.head(5)

326570    2
292150    2
252556    2
401470    2
260022    2
Name: Name, dtype: int64

In [13]:
index_sorted_by_length = lengths_sorted_by_length.index
index_sorted_by_length

Int64Index([326570, 292150, 252556, 401470, 260022, 281052,  89716,  95228,
            223306, 378237,
            ...
            327472, 316886, 334166, 337301, 321913, 339472, 327358, 337477,
            312543, 321792],
           dtype='int64', length=407428)

In [ ]:
# now pass the index to loc. This is yet another way 
# that loc can be used that we did not discuss in lecture!
babynames.loc[index_sorted_by_length].head(5)